In [1]:
import opfython.math.general as g
import opfython.stream.loader as l
import opfython.stream.parser as p
import opfython.stream.splitter as s

from downSampling_OPF import DS
import os

import numpy as np
import sys

import logging
logging.disable(sys.maxsize)
np.set_printoptions(threshold=sys.maxsize)

In [2]:
datasets = ['1069_5gt','1069_7gt','1086_5ge', '1086_7ge','1143_5gte','1143_7gte']
folds = np.arange(1,21)

dsOPF = DS()



for dsds in range(len(datasets)):
    ds = datasets[dsds]
    for ff in range(len(folds)): 
        f = folds[ff]
        train = np.loadtxt('data/{}/{}/train.txt'.format(ds,f),delimiter=',', dtype=np.float32)
        valid = np.loadtxt('data/{}/{}/valid.txt'.format(ds,f),delimiter=',', dtype=np.float32)
        test = np.loadtxt('data/{}/{}/test.txt'.format(ds,f),delimiter=',', dtype=np.float32)
        
        concat = np.concatenate((train, valid))
        X = concat[:,:-1]
        Y = concat[:,-1].astype(np.int) 
        indices = np.arange(len(X))
        
        output = dsOPF.run(X, Y, indices)
        
        X = X[:len(train),...]
        Y = Y[:len(train),...]
        output = output[:len(train),...]
        
        X_test = test[:,:-1]
        Y_test = test[:,-1].astype(np.int) 
        
        
        pathDataset = 'data/{}/{}'.format(ds,f)
        if not os.path.exists(pathDataset):
            os.makedirs(pathDataset)  
            
            
            
        #output_order = output[output[:,1]==majority_class]
        #output_majority_negative = output_majority[output_majority[:,2]<0]
        
        #X_train = np.delete(X, output_majority_negative[:,0],0)
        #Y_train = np.delete(Y, output_majority_negative[:,0])
        #self.__saveDataset(X_train,Y_train, pathDataset,fmt,'major_negative')
        #self.__saveResults(X_train,Y_train, X_test, Y_test, ds,f, 'major_negative')
            
            
        
        #1st case: remove samples from majoritary class with negative scores      
        #dsOPF.major_negative( output, X, Y, pathDataset, 1)

        #2st case: remove samples from majoritary class with negative or zero scores
        #dsOPF.major_neutral( output, X, Y, pathDataset, 1)   
        
        #3st case: remove all samples with negative
        #dsOPF.negative( output, X, Y, pathDataset, 1)
        
        #4st case: remove samples from majoritary class with negative or zero scores 
            # and from minoritary class with negative scores
        #dsOPF.negatives_major_zero( output, X, Y, pathDataset, 1)
        
        #5st case: remove samples from majoritary class until balancing the dataset
        dsOPF.balance( output, X, Y, X_test, Y_test, pathDataset, 1, ds,f)
        